In [2]:
import sys
sys.path.append('./yolo')
from detection_fusion import EnsembleModel
import numpy as np
import matplotlib.pyplot as plt
from rcnn.dataset import ChestCocoDetection
import matplotlib.patches as patches
from torch.utils.data import DataLoader
import tqdm
import torch

In [3]:
from yolo.utils.general import xywh2xyxy, scale_coords, box_iou
from yolo.utils.metrics import ap_per_class
import torch
from rcnn.model import ChestRCNN
from yolo.yolo import Model
from yolo.utils.datasets import LoadImages
from yolo.utils.general import check_img_size

In [4]:
YOLO_FINAL_MODEL_PATH = "./yolo/models_final/yolov5_epoch_26.pt"
FASTER_RCNN_FINAL_MODEL_PATH = "./rcnn/models/fasterrcnn_epoch_23.pt"
RESNET_BACKBONE_PATH = "./resnet/models/resnext101_32x8d_epoch_35.pt"

yolov5_weights = torch.load(YOLO_FINAL_MODEL_PATH)
fasterrcnn_r101_weights = torch.load(FASTER_RCNN_FINAL_MODEL_PATH)

yolo = Model(cfg="./yolo/yolo5l.yaml",ch=3,nc=1)
yolo.load_state_dict(yolov5_weights, strict=False) 

fasterRcnn = ChestRCNN(RESNET_BACKBONE_PATH)
fasterRcnn.load_state_dict(fasterrcnn_r101_weights)
ensemble = EnsembleModel(fasterRcnn=fasterRcnn, yolo=yolo)

D:\Anaconda\envs\aml\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


### Evaluation of Ensemble method

In [5]:
def collate_fn(batch):
    imgs, targets, path = zip(*batch)
    imgs = torch.stack(imgs)
    return tuple((imgs, targets, path))

In [6]:
# need bs = 1 because model fusion cannot handle batches 
test = ChestCocoDetection(root="F:\\aml-project\data\\siim-covid19-detection", ann_file="F:\\aml-project\\data\\siim-covid19-detection\\test.json", training=False, image_size=512, detection_fusion=True)
test_loader = DataLoader(test, batch_size=1, shuffle=False, pin_memory=False, num_workers=0, collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [7]:
VISUALIZATION = True
jdict, stats, ap, ap_class = [], [], [], []
stats_yolo, stats_fcnn = [], []
iouv = torch.linspace(0.5, 0.55, 1) # iou vector for mAP@0.5:0.95
niou = iouv.numel()
general_test_results = []
selected_samples = [5, 10, 18, 32, 26]
visualization_sample_data = {"img": [], "gt_box": [], "yolo_box": [], "yolo_scores": [], "fcnn_box": [], "fcnn_scores": [], "ensemble_box": [], "ensemble_scores": [] }
for i, (images, targets, path) in enumerate(tqdm.tqdm(test_loader)):
    nb, _, height, width = images.shape
    assert nb == 1

    # Load image with Yolo transformations since we use the FCNN data loader per default
    path = '.\..\\data\\siim-covid19-detection\\' + path[0]
    stride = int(yolo.stride.max())  # model stride
    img_size = check_img_size(512, s=stride)  # check img_size
    yolo_img = LoadImages(path,img_size=img_size,stride=stride).__iter__().__next__()[1]
    yolo_img = torch.from_numpy(yolo_img).float()
    yolo_img /= 255.0
    if yolo_img.ndimension() == 3:
         yolo_img = yolo_img.unsqueeze(0)

    _, boxes, scores, pred_labels, yolo_box, yolo_scores, yolo_labels, fcnn_box, fcnn_scores, fcnn_labels = ensemble.detection_fusion(images,yolo_img, extended_output=True)
    boxes = torch.tensor(boxes)
    yolo_box = torch.tensor(yolo_box)
    fcnn_box = torch.tensor(fcnn_box)
    gt_boxes = targets[0]['boxes']
    labels = targets[0]['labels']
    nl = len(labels)
    tcls = labels[:].tolist() if nl else []  # target class

    # save one sample prediction for visualization
    if len(boxes) and i in selected_samples :
        visualization_sample_data["img"].append(images)
        visualization_sample_data["gt_box"].append(gt_boxes)
        visualization_sample_data["yolo_box"].append(yolo_box)
        visualization_sample_data["yolo_scores"].append(yolo_scores)
        visualization_sample_data["fcnn_box"].append(fcnn_box)
        visualization_sample_data["fcnn_scores"].append(fcnn_scores)
        visualization_sample_data["ensemble_box"].append(boxes)
        visualization_sample_data["ensemble_scores"].append(scores)
        
        if VISUALIZATION and len(selected_samples) == len(visualization_sample_data["img"]):
            break


    if len(boxes) == 0:
        if nl:
            stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
            stats_yolo.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
            stats_fcnn.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))

        continue

    correct = torch.zeros(len(boxes), niou, dtype=torch.bool)
    correct_yolo = torch.zeros(len(yolo_box), niou, dtype=torch.bool)
    correct_fcnn = torch.zeros(len(fcnn_box), niou, dtype=torch.bool)
    
    if nl:
        detected = []
        detected_yolo = []
        detected_fcnn = []
        tcls_tensor = labels[:]

        # Per target class
        for cls in torch.tensor([1]):
            ti = (cls == tcls_tensor).nonzero(as_tuple=False).view(-1)
            pi = (cls == torch.tensor(pred_labels)).nonzero(as_tuple=False).view(-1)
            pi_yolo = (cls == torch.tensor(yolo_labels)).nonzero(as_tuple=False).view(-1)
            pi_fcnn = (cls == torch.tensor(fcnn_labels)).nonzero(as_tuple=False).view(-1)

            # Search for detections
            if pi.shape[0]:        
                ious, i = box_iou(boxes[pi], gt_boxes[ti]).max(1)  # best ious, indices
                # Append detections - Ensemble
                detected_set = set()
                for j in (ious > iouv[0]).nonzero(as_tuple=False):
                    d = ti[i[j]]  
                    if d.item() not in detected_set:
                        detected_set.add(d.item())
                        detected.append(d)
                        correct[pi[j]] = ious[j] > iouv  
                        if len(detected) == nl: 
                            break
                            
            if pi_yolo.shape[0]:
                ious_yolo, i_yolo = box_iou(yolo_box[pi_yolo], gt_boxes[ti]).max(1)
                # Yolo
                detected_set_yolo = set()
                for j in (ious_yolo > iouv[0]).nonzero(as_tuple=False):
                    d = ti[i_yolo[j]]  
                    if d.item() not in detected_set_yolo:
                        detected_set_yolo.add(d.item())
                        detected_yolo.append(d)
                        correct_yolo[pi_yolo[j]] = ious_yolo[j] > iouv  
                        if len(detected_yolo) == nl: 
                            break
                            
            if pi_fcnn.shape[0]:
                ious_fcnn, i_fcnn = box_iou(fcnn_box[pi_fcnn], gt_boxes[ti]).max(1)
                # Faster R-CNN
                detected_set_fcnn = set()
                for j in (ious_fcnn > iouv[0]).nonzero(as_tuple=False):
                    d = ti[i_fcnn[j]]  
                    if d.item() not in detected_set_fcnn:
                        detected_set_fcnn.add(d.item())
                        detected_fcnn.append(d)
                        correct_fcnn[pi_fcnn[j]] = ious_fcnn[j] > iouv  
                        if len(detected_fcnn) == nl: 
                            break



        # Append statistics (correct, conf, pcls, tcls)
        stats.append((correct.cpu(), torch.tensor(scores),torch.tensor(pred_labels), tcls))
        stats_yolo.append((correct_yolo.cpu(), torch.tensor(yolo_scores),torch.tensor(yolo_labels), tcls))
        stats_fcnn.append((correct_fcnn.cpu(), torch.tensor(fcnn_scores),torch.tensor(fcnn_labels), tcls))

for statistic in [stats, stats_yolo, stats_fcnn]:
    statistic = [np.concatenate(x, 0) for x in zip(*statistic)] 
    if len(statistic) and statistic[0].any():
        p, r, ap, f1, ap_class = ap_per_class(*statistic)
        ap50, ap = ap[:, 0], ap.mean(1) 
        mp, mr, map50, m = p.mean(), r.mean(), ap50.mean(), ap.mean()
        nt = np.bincount(statistic[3].astype(np.int64), minlength=1)

        general_test_results.append({
        "precision": p,
        "recall": r,
        "ap": ap,
        "f1": f1,
        "ap_class": ap_class,
        "ap": ap,
        "ap50": ap50,
        "mp": mp,
        "mr": mr,
        "map50": map50,
        "map": m
    })

    else:
        nt = torch.zeros(1)


  4%|▎         | 32/859 [02:10<56:22,  4.09s/it]


In [ ]:
# Ensemble
general_test_results[0]

{'precision': array([    0.59357]),
 'recall': array([     0.5302]),
 'ap': array([    0.45424]),
 'f1': array([     0.5601]),
 'ap_class': array([1]),
 'ap50': array([    0.45424]),
 'mp': 0.5935737555211111,
 'mr': 0.5301970756516211,
 'map50': 0.454241758431956,
 'map': 0.454241758431956}

In [19]:
# YOLO
general_test_results[1]

{'precision': array([    0.50593]),
 'recall': array([    0.45048]),
 'ap': array([    0.34455]),
 'f1': array([    0.47659]),
 'ap_class': array([1]),
 'ap50': array([    0.34455]),
 'mp': 0.505925645110543,
 'mr': 0.4504762082687064,
 'map50': 0.34455343562825014,
 'map': 0.34455343562825014}

In [20]:
# Faster R-CNN
general_test_results[2]

{'precision': array([    0.60309]),
 'recall': array([    0.50328]),
 'ap': array([    0.40929]),
 'f1': array([    0.54868]),
 'ap_class': array([1]),
 'ap50': array([    0.40929]),
 'mp': 0.6030946457346565,
 'mr': 0.5032776105936924,
 'map50': 0.40928837471026075,
 'map': 0.40928837471026075}

In [8]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
        return tensor


unorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
imgs = []
for i in visualization_sample_data['img']:
    imgs.append(unorm(i))

In [12]:
import matplotlib.gridspec as gridspec
nrow = 5
ncol = 4

def add_bounding_boxes(boxes, ax):
    for box in boxes:
        mp_box = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], edgecolor="r", facecolor='none')
        ax.add_patch(mp_box)

def show_samples_for(gt, yolo, fcnn, ensemble):
    fig = plt.figure(figsize=(6, 10), constrained_layout=True) 
    fig.subplots_adjust(wspace=0, hspace=0)
    gs = gridspec.GridSpec(5, 4, width_ratios=[1, 1, 1, 1],
         wspace=0.1, hspace=0.1) 

    for im_i, (img, box, name) in enumerate([gt, yolo, fcnn, ensemble]):
        ax = plt.subplot(gs[0, im_i])
        ax.set_title(name)
        for im_j, (i, b) in enumerate(zip(img, box)):
            ax = plt.subplot(gs[im_j,im_i])
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.imshow(i.squeeze_(0).permute(1,2,0))
            add_bounding_boxes(b, ax)           

    fig.savefig("combo.png",bbox_inches='tight', dpi=200)


show_samples_for(
    (imgs, visualization_sample_data['gt_box'], "Ground Truth"), 
    (imgs, visualization_sample_data['yolo_box'], "YOLO"),
    (imgs, visualization_sample_data['fcnn_box'], "Faster R-CNN"),
    (imgs, visualization_sample_data['ensemble_box'], "Ensemble")
)

C:\Users\Julian\AppData\Local\Temp/ipykernel_18872/2475878709.py:12: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  fig.subplots_adjust(wspace=0, hspace=0)
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for f